# Introduction #
This code has the following functions:
1. A new version of deep convolutional neural network model (ResNet) for binary classfication based on the base project COVID-CXR
2. Generate a h5 model file with DNN_RESNET model

# Load Libraries

In [ ]:
import pandas as pd
import yaml
import os
import datetime
import random
import cv2
import dill
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.summary as tf_summary
from imblearn.over_sampling import RandomOverSampler
from math import ceil
from tensorflow.keras.metrics import BinaryAccuracy, CategoricalAccuracy, Precision, Recall, AUC
from tensorflow.keras.models import save_model
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorboard.plugins.hparams import api as hp

from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input, MaxPool2D, Conv2D, Flatten, LeakyReLU, BatchNormalization, Activation, concatenate, GlobalAveragePooling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.initializers import Constant
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, ResNet101V2

from tensorflow.keras.metrics import Metric
from tensorflow.python.keras.utils import metrics_utils
from tensorflow.python.ops import init_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.keras.utils.generic_utils import to_list
from tensorflow.python.keras import backend as K

# Define functions that will be used in the main program
This following cell was a refined version in order to run on Google colab. 

Major adjustments I did for these code comparing with the orignial code is:

1. Adjust the code to run on Mac and Ubuntu since the original code is developed under Windows and its style for directory creates difficulty to run in other Operation Systems;
2. Adjust the code to run with the new version of Tensorflow. The main change is replace fit_generator(...) with fit(...) because fit_generator(...) was depreciated by the new version;
3. Fix other mistakes when migrate to run on Google Colab. (The original code was run on command line)

The original code can be found by the following link: https://github.com/aildnont/covid-cxr.
* The following cell contains useful functions from the above link to generate the model file

In [ ]:
class F1Score(Metric):
    '''
    Custom tf.keras metric that calculates the F1 Score
    '''

    def __init__(self, thresholds=None, top_k=None, class_id=None, name=None, dtype=None):
        '''
        Creates an instance of the  F1Score class
        :param thresholds: A float value or a python list/tuple of float threshold values in [0, 1].
        :param top_k: An int value specifying the top-k predictions to consider when calculating precision
        :param class_id: Integer class ID for which we want binary metrics. This must be in the half-open interval
                `[0, num_classes)`, where `num_classes` is the last dimension of predictions
        :param name: string name of the metric instance
        :param dtype: data type of the metric result
        '''
        super(F1Score, self).__init__(name=name, dtype=dtype)
        self.init_thresholds = thresholds
        self.top_k = top_k
        self.class_id = class_id

        default_threshold = 0.5 if top_k is None else metrics_utils.NEG_INF
        self.thresholds = metrics_utils.parse_init_thresholds(
            thresholds, default_threshold=default_threshold)
        self.true_positives = self.add_weight('true_positives', shape=(len(self.thresholds),),
                                              initializer=init_ops.zeros_initializer)
        self.false_positives = self.add_weight('false_positives', shape=(len(self.thresholds),),
                                               initializer=init_ops.zeros_initializer)
        self.false_negatives = self.add_weight('false_negatives', shape=(len(self.thresholds),),
                                               initializer=init_ops.zeros_initializer)

    def update_state(self, y_true, y_pred, sample_weight=None):
        '''
        Accumulates true positive, false positive and false negative statistics.
        :param y_true: The ground truth values, with the same dimensions as `y_pred`. Will be cast to `bool`
        :param y_pred: The predicted values. Each element must be in the range `[0, 1]`
        :param sample_weight: Weighting of each example. Defaults to 1. Can be a `Tensor` whose rank is either 0,
               or the same rank as `y_true`, and must be broadcastable to `y_true`
        :return: Update operation
        '''
        metrics_utils.update_confusion_matrix_variables(
            {
                metrics_utils.ConfusionMatrix.TRUE_POSITIVES: self.true_positives,
                metrics_utils.ConfusionMatrix.FALSE_POSITIVES: self.false_positives,
                metrics_utils.ConfusionMatrix.FALSE_NEGATIVES: self.false_negatives
            },
            y_true, y_pred, thresholds=self.thresholds, top_k=self.top_k, class_id=self.class_id,
            sample_weight=sample_weight)


    def result(self):
        '''
        Compute the value for the F1 score. Calculates precision and recall, then F1 score.
        F1 = 2 * precision * recall / (precision + recall)
        :return: F1 score
        '''
        precision = math_ops.div_no_nan(self.true_positives, self.true_positives + self.false_positives)
        recall = math_ops.div_no_nan(self.true_positives, self.true_positives + self.false_negatives)
        result = math_ops.div_no_nan(2 * precision * recall, precision + recall)
        return result[0] if len(self.thresholds) == 1 else result

    def reset_states(self):
        '''
        Resets all of the metric state variables. Called between epochs, when a metric is evaluated during training.
        '''
        num_thresholds = len(to_list(self.thresholds))
        K.batch_set_value(
            [(v, np.zeros((num_thresholds,))) for v in self.variables])

    def get_config(self):
        '''
        Returns the serializable config of the metric.
        :return: serializable config of the metric
        '''
        config = {
            'thresholds': self.init_thresholds,
            'top_k': self.top_k,
            'class_id': self.class_id
        }
        base_config = super(F1Score, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))



def remove_text(img):
    '''
    Attempts to remove bright textual artifacts from X-ray images. For example, many images indicate the right side of
    the body with a white 'R'. Works only for very bright text.
    :param img: Numpy array of image
    :return: Array of image with (ideally) any characters removed and inpainted
    '''
    mask = cv2.threshold(img, 230, 255, cv2.THRESH_BINARY)[1][:, :, 0].astype(np.uint8)
    img = img.astype(np.uint8)
    result = cv2.inpaint(img, mask, 10, cv2.INPAINT_NS).astype(np.float32)
    return result


def get_class_weights(histogram, class_multiplier=None):
    '''
    Computes weights for each class to be applied in the loss function during training.
    :param histogram: A list depicting the number of each item in different class
    :param class_multiplier: List of values to multiply the calculated class weights by. For further control of class weighting.
    :return: A dictionary containing weights for each class
    '''
    weights = [None] * len(histogram)
    for i in range(len(histogram)):
        weights[i] = (1.0 / len(histogram)) * sum(histogram) / histogram[i]
    class_weight = {i: weights[i] for i in range(len(histogram))}
    if class_multiplier is not None:
        class_weight = [class_weight[i] * class_multiplier[i] for i in range(len(histogram))]
    print("Class weights: ", class_weight)
    return class_weight


def random_minority_oversample(train_set):
    '''
    Oversample the minority class using the specified algorithm
    :param train_set: Training set image file names and labels
    :return: A new training set containing oversampled examples
    '''
    X_train = train_set[[x for x in train_set.columns if x != 'label']].to_numpy()
    if X_train.shape[1] == 1:
        X_train = np.expand_dims(X_train, axis=-1)
    Y_train = train_set['label'].to_numpy()
    sampler = RandomOverSampler(random_state=np.random.randint(0, high=1000))
    X_resampled, Y_resampled = sampler.fit_resample(X_train, Y_train)
    filenames = X_resampled[:, 1]     # Filename is in second column
    label_strs = X_resampled[:, 2]    # Class name is in second column
    print("Train set shape before oversampling: ", X_train.shape, " Train set shape after resampling: ", X_resampled.shape)
    train_set_resampled = pd.DataFrame({'filename': filenames, 'label': Y_resampled, 'label_str': label_strs})
    return train_set_resampled


def train_model(cfg, data, callbacks, verbose=1):
    '''
    Train a and evaluate model on given data.
    :param cfg: Project config (from config.yml)
    :param data: dict of partitioned dataset
    :param callbacks: list of callbacks for Keras model
    :param verbose: Verbosity mode to pass to model.fit_generator()
    :return: Trained model and associated performance metrics on the test set
    '''

    # Create ImageDataGenerators
    train_img_gen = ImageDataGenerator(rotation_range=10, preprocessing_function=remove_text,
                                       samplewise_std_normalization=True, samplewise_center=True)
    val_img_gen = ImageDataGenerator(preprocessing_function=remove_text,
                                       samplewise_std_normalization=True, samplewise_center=True)
    test_img_gen = ImageDataGenerator(preprocessing_function=remove_text,
                                       samplewise_std_normalization=True, samplewise_center=True)

    # Create DataFrameIterators
    img_shape = tuple([224, 224])
    y_col = 'label_str'
    class_mode = 'categorical'
    train_generator = train_img_gen.flow_from_dataframe(dataframe=data['TRAIN'], directory=covid_cxr + cfg['PATHS']['RAW_DATA'],
        x_col="filename", y_col=y_col, target_size=img_shape, batch_size=cfg['TRAIN']['BATCH_SIZE'],
        class_mode=class_mode, validate_filenames=False)
    val_generator = val_img_gen.flow_from_dataframe(dataframe=data['VAL'], directory=covid_cxr + cfg['PATHS']['RAW_DATA'],
        x_col="filename", y_col=y_col, target_size=img_shape, batch_size=cfg['TRAIN']['BATCH_SIZE'],
        class_mode=class_mode, validate_filenames=False)
    test_generator = test_img_gen.flow_from_dataframe(dataframe=data['TEST'], directory=covid_cxr + cfg['PATHS']['RAW_DATA'],
        x_col="filename", y_col=y_col, target_size=img_shape, batch_size=cfg['TRAIN']['BATCH_SIZE'],
        class_mode=class_mode, validate_filenames=False, shuffle=False)

    # Apply class imbalance strategy. We have many more X-rays negative for COVID-19 than positive.
    histogram = np.bincount(np.array(train_generator.labels).astype(int))  # Get class distribution
    class_weight = None
    if cfg['TRAIN']['IMB_STRATEGY'] == 'class_weight':
        class_multiplier = cfg['TRAIN']['CLASS_MULTIPLIER']
        class_multiplier = [class_multiplier[cfg['DATA']['CLASSES'].index(c)] for c in test_generator.class_indices]
        class_weight = get_class_weights(histogram, class_multiplier)

    # Define metrics.
    covid_class_idx = test_generator.class_indices['COVID-19']   # Get index of COVID-19 class
    thresholds = 1.0 / len(cfg['DATA']['CLASSES'])      # Binary classification threshold for a class
    metrics = [CategoricalAccuracy(name='accuracy'),
               Precision(name='precision', thresholds=thresholds, class_id=covid_class_idx),
               Recall(name='recall', thresholds=thresholds, class_id=covid_class_idx),
               AUC(name='auc'),
               F1Score(name='f1score', thresholds=thresholds, class_id=covid_class_idx)]

    # Define the model.
    print('Training distribution: ', ['Class ' + list(test_generator.class_indices.keys())[i] + ': ' + str(histogram[i]) + '. '
           for i in range(len(histogram))])
    input_shape = cfg['DATA']['IMG_DIM'] + [3]

    histogram = np.bincount(data['TRAIN']['label'].astype(int))
    output_bias = np.log([histogram[i] / (np.sum(histogram) - histogram[i]) for i in range(histogram.shape[0])])
    model = dcnn_resnet_binary(input_shape, metrics, output_bias=output_bias)

    # Train the model.
    steps_per_epoch = ceil(train_generator.n / train_generator.batch_size)
    val_steps = ceil(val_generator.n / val_generator.batch_size)
    history = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=cfg['TRAIN']['EPOCHS'],
                                  validation_data=val_generator, validation_steps=val_steps, callbacks=callbacks,
                                  verbose=verbose)
    # Run the model on the test set and print the resulting performance metrics.
    test_results = model.evaluate_generator(test_generator, verbose=1)
    test_metrics = {}
    test_summary_str = [['**Metric**', '**Value**']]
    for metric, value in zip(model.metrics_names, test_results):
        test_metrics[metric] = value
        print(metric, ' = ', value)
        test_summary_str.append([metric, str(value)])
    return model, test_metrics, test_generator



# Define my own version of of DCNN_RESNET. #
I defined a function dcnn_resnet_binary based on the original code. The main change is I used some new hyperparameters. 

It is not difficult to create more models like ResNet50V2, ResNet101V2, VGG16, and etc. All of them can generate different h5 model files for future use.

In [ ]:
def dcnn_resnet_binary(input_shape, metrics, output_bias=None):
    '''
    Defines a deep convolutional neural network model for binary X-ray classification.
    :param model_config: A dictionary of parameters associated with the model architecture
    :param input_shape: The shape of the model input
    :param metrics: Metrics to track model's performance
    :return: a Keras Model object with the architecture defined in this method
    '''

    # Set hyperparameters
    nodes_dense0 = 128
    lr = 0.00001
    dropout = 0.4
    l2_lambda = 0.0001
    optimizer = Adam(learning_rate=lr)
    
    init_filters = 16
    filter_exp_base = 3
    conv_blocks = 3
    kernel_size = (3,3)
    max_pool_size = (2,2)
    strides = (1,1)

    # Set output bias
    if output_bias is not None:
        output_bias = Constant(output_bias)

    # Input layer
    X_input = Input(input_shape)
    X = X_input

    # Add convolutional (residual) blocks
    for i in range(conv_blocks):
        X_res = X
        X = Conv2D(init_filters * (filter_exp_base ** i), kernel_size, strides=strides, padding='same',
                         kernel_initializer='he_uniform', activity_regularizer=l2(l2_lambda),
                         name='conv' + str(i) + '_0')(X)
        X = BatchNormalization()(X)
        X = LeakyReLU()(X)
        X = Conv2D(init_filters * (filter_exp_base ** i), kernel_size, strides=strides, padding='same',
                         kernel_initializer='he_uniform', activity_regularizer=l2(l2_lambda),
                         name='conv' + str(i) + '_1')(X)
        X = concatenate([X, X_res], name='concat' + str(i))
        X = BatchNormalization()(X)
        X = LeakyReLU()(X)
        X = MaxPool2D(max_pool_size, padding='same')(X)

    # Add fully connected layers
    X = Flatten()(X)
    X = Dropout(dropout)(X)
    X = Dense(nodes_dense0, kernel_initializer='he_uniform', activity_regularizer=l2(l2_lambda))(X)
    X = LeakyReLU()(X)
    X = Dense(2, bias_initializer=output_bias)(X)
    Y = Activation('softmax', dtype='float32', name='output')(X)

    # Set model loss function, optimizer, metrics.
    model = Model(inputs=X_input, outputs=Y)
    model.summary()

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=metrics)
    return model


# Preparing updated covid-cxr #

This project is based on the previous work from https://github.com/aildnont/covid-cxr. However, the original code is developed under windows which do not work on a Mac. It also cannot run in the python environment other than version 3.6. It only works with tensorflow_gpu and etc.

I fixed most of the issues and did the following work for our project:
1. Preprocess the data and generate train, test, validation, and store this information in three separate files (covid-cxr/data/processed/train_set.csv, covid-cxr/data/processed/test_set.csv, covid-cxr/data/processed/val_set.csv);
2. Modify the config.yml which defines the parameters;
3. Store all the images files which used to predict;
4. Generate the model files and store them in the following directory (covid-cxr/results/models).


To make it easy, I shared all codes, data, and models with the public, and you can access them through the below link.
https://drive.google.com/drive/folders/1wyvVPx9TvArGIldJkDpQfLRufwKZOTKQ?usp=sharing

You have two ways to use it:
1. The fast way to use it is as below:
Step 1. Login to your google drive and paste the above link to your browser;
Step 2. Right-click the folder name *covid-cxr* and click *Add shortcut to Drive*. Now, if you connect your google drive in colab, you will see the covid-cxr directory has been linked in your home.

The code connects your google drive in colab environment:

```
from google.colab import drive
drive.mount('/content/drive')
```

2. You can also choose to download the whole directory as a zip file, and upload it to your google colab directory. Since it is huge (3GB), it could be slow.

#Load model and config file#
In order to reproduce the result, you will need to use your own directory to load model and data. Please make sure you finish what I describe in the Preparing updated covid-cxr section before run these codes.



In [ ]:
# Load gdrive since all data is stored on gdrive
from google.colab import drive
drive.mount('/content/drive')
covid_cxr = '/content/drive/MyDrive/covid-cxr/'
import yaml
from pathlib import Path

cfg = yaml.safe_load(open(Path(covid_cxr) / 'config.yml', 'r'))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load test set, callback and generate h5 file

We collected a very large dataset containing chest x-ray image files. To balance accuracy and time to run it, we set Epoch to 5.

When the training finished, a file named "covid.h5" will be generated. 


In [ ]:
# Load dataset file paths and labels
data = {}
data['TRAIN'] = pd.read_csv(covid_cxr + cfg['PATHS']['TRAIN_SET'])
data['VAL'] = pd.read_csv(covid_cxr + cfg['PATHS']['VAL_SET'])
data['TEST'] = pd.read_csv(covid_cxr + cfg['PATHS']['TEST_SET'])

# Set callbacks.
early_stopping = EarlyStopping(monitor='val_loss', verbose=1, patience=7, mode='min', restore_best_weights=True)
callbacks = [early_stopping]

# Conduct the desired train model
model, test_metrics, test_generator = train_model(cfg, data, callbacks)


Found 1489 non-validated image filenames belonging to 2 classes.
Found 146 non-validated image filenames belonging to 2 classes.
Found 192 non-validated image filenames belonging to 2 classes.
Class weights:  [26.589285714285715, 0.07643737166324435]
Training distribution:  ['Class COVID-19: 28. ', 'Class non-COVID-19: 1461. ']
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv0_0 (Conv2D)               (None, 224, 224, 16  448         ['input_2[0][0]']                
                                )                          

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2034: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


47/47 [==============================] - 144s 3s/step - loss: 451.0175 - accuracy: 0.9476 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.9610 - f1score: 0.0000e+00 - val_loss: 358.3711 - val_accuracy: 0.8699 - val_precision: 0.0556 - val_recall: 0.3333 - val_auc: 0.9105 - val_f1score: 0.0952
Epoch 2/5
47/47 [==============================] - 81s 2s/step - loss: 419.7025 - accuracy: 0.9805 - precision: 0.4286 - recall: 0.1071 - auc: 0.9868 - f1score: 0.1714 - val_loss: 305.6641 - val_accuracy: 0.5753 - val_precision: 0.0317 - val_recall: 0.6667 - val_auc: 0.6435 - val_f1score: 0.0606
Epoch 3/5
47/47 [==============================] - 81s 2s/step - loss: 395.2088 - accuracy: 0.9738 - precision: 0.1765 - recall: 0.1071 - auc: 0.9885 - f1score: 0.1333 - val_loss: 291.6352 - val_accuracy: 0.8219 - val_precision: 0.0400 - val_recall: 0.3333 - val_auc: 0.9144 - val_f1score: 0.0714
Epoch 4/5
47/47 [==============================] - 81s 2s/step - loss: 374.3308 - accuracy: 0.9738 - preci

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:198: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.


6/6 [==============================] - 67s 13s/step - loss: 294.5326 - accuracy: 0.9375 - precision: 0.4000 - recall: 0.6667 - auc: 0.9772 - f1score: 0.5000
loss  =  294.5326232910156
accuracy  =  0.9375
precision  =  0.4000000059604645
recall  =  0.6666666865348816
auc  =  0.9772406220436096
f1score  =  0.5


In [ ]:
# Generate h5 file
model_path = 'covid.h5'
save_model(model, model_path)  # Save the model

print("The generated file is {}".format(model_path))

The generated file is covid.h5


#Credits#

1.   https://github.com/aildnont/covid-cxr


